In [2]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import seaborn as sns
from standardization import calculate_standardization_from_files

In [ ]:
files = glob.glob("/scratch/users/schuetzn/data/mhc_dataset/*/metadata.parquet")
standardization_df = calculate_standardization_from_files(files)
standardization_df.to_csv("/scratch/users/schuetzn/data/mhc_dataset_out/standardization_params.csv")

In [ ]:
standardization_df = pd.read_csv("/scratch/users/schuetzn/data/mhc_dataset_out/standardization_params.csv")
standardization_df

In [2]:
dfs = []
files = glob.glob("/scratch/users/schuetzn/data/mhc_dataset/*/metadata.parquet")
for file in files:
    hq = file.split("/")[-2]
    df = pd.read_parquet(file)
    df["healthCode"] = hq
    dfs.append(df)

In [3]:
df = pd.concat(dfs)

In [ ]:
df

In [4]:
daily_stats_per_modality = []
for idx, row in df.groupby(["healthCode", "date"]):
    # Common data for all entries
    healthCode = row["healthCode"].iloc[0]
    date = row["date"].iloc[0]
    
    # Map specific variables to their corresponding indices in the dataframe
    variable_mappings = {
        "WorkoutsAny": [14, 15, 16, 17, 18, 19, 20, 21, 22, 23],  # All workout types
        "WorkoutsRunning": [16],  # HKWorkoutActivityTypeRunning
        "WorkoutsCycling": [15],  # HKWorkoutActivityTypeCycling
        "HeartRate": [5],  # HKQuantityTypeIdentifierHeartRate
        "StepsTaken": [0],  # HKQuantityTypeIdentifierStepCount
        "DistanceWalkingRunning": [2],  # HKQuantityTypeIdentifierDistanceWalkingRunning
        "DistanceCycling": [3],  # HKQuantityTypeIdentifierDistanceCycling
        "ActiveCaloriesBurned": [1],  # HKQuantityTypeIdentifierActiveEnergyBurned
        "SleepAny": [12, 13]  # Both sleep types - Asleep and InBed
    }
    
    # Create entry for this healthCode/date combination
    entry = {
        "healthCode": healthCode,
        "date": date
    }
    
    # Calculate data_coverage_sum for each requested variable
    for variable_name, indices in variable_mappings.items():
        # For variables that map to multiple indices, sum the data_coverage across those indices
        if len(indices) > 1:
            coverage_sum = row.iloc[indices].data_coverage.sum()
        else:
            coverage_sum = row.iloc[indices[0]].data_coverage.sum()
        
        entry[f"{variable_name}_coverage_sum"] = coverage_sum
    
    daily_stats_per_modality.append(entry)

In [ ]:
len(daily_stats_per_modality)

In [2]:
import pickle
#with open("../per_modality_daily_v2.pkl", "wb") as f:
#   pickle.dump(daily_stats_per_modality, f)
with open("../per_modality_daily_v2.pkl", "rb") as f:
     daily_stats_per_modality = pickle.load(f)

In [3]:
coverage_by_modality_df = pd.DataFrame(daily_stats_per_modality)

In [ ]:
coverage_by_modality_df


In [ ]:
# Create a dictionary to store the results
modality_stats = {}

# Get the list of modalities (excluding healthCode and date columns)
modalities = [col for col in coverage_by_modality_df.columns if col not in ['healthCode', 'date']]

# For each modality, calculate days > 0, unique participants, and estimated data points
for modality in modalities:
    # Filter to only rows where coverage > 0 for this modality
    covered_data = coverage_by_modality_df[coverage_by_modality_df[modality] > 0]
    
    # Count unique days and participants
    days_count = len(covered_data)
    participants_count = covered_data['healthCode'].nunique()
    
    # Calculate estimated total data points (coverage of 100 = 1440 data points)
    total_coverage = coverage_by_modality_df[modality].sum()
    estimated_data_points = total_coverage * (1440/100)
    
    # Store in results dictionary
    modality_stats[modality] = {
        'days_with_coverage': days_count,
        'unique_participants': participants_count,
        'estimated_data_points': int(estimated_data_points)  # Convert to integer for cleaner display
    }

# Convert to a DataFrame for better display
result_df = pd.DataFrame.from_dict(modality_stats, orient='index')

# Rename the modalities for clarity
modality_names = {
    'WorkoutsAny_coverage_sum': 'Workouts (Any Type)',
    'WorkoutsRunning_coverage_sum': 'Workouts - Running',
    'WorkoutsCycling_coverage_sum': 'Workouts - Cycling',
    'HeartRate_coverage_sum': 'Heart Rate',
    'StepsTaken_coverage_sum': 'Steps Taken',
    'DistanceWalkingRunning_coverage_sum': 'Distance Walking/Running',
    'DistanceCycling_coverage_sum': 'Distance Cycling',
    'ActiveCaloriesBurned_coverage_sum': 'Active Calories Burned',
    'SleepAny_coverage_sum': 'Sleep (Any Type)'
}

# Create a new dataframe with clear modality names
clear_result_df = result_df.copy()
clear_result_df.index = [modality_names[idx] for idx in clear_result_df.index]

# Format the estimated data points with commas for readability
clear_result_df['estimated_data_points'] = clear_result_df['estimated_data_points'].apply(lambda x: f"{x:,}")

# Display the results
print("Coverage statistics for each modality:")
print(clear_result_df)

# Optionally, sort by number of days with coverage to see most common modalities
print("\nModalities sorted by coverage (most to least):")
print(clear_result_df.sort_values('days_with_coverage', ascending=False))

In [10]:
stats = df.groupby("healthCode")["date"].apply(lambda x: len(np.unique(x)))

In [ ]:
counts_series = coverage_df.groupby("healthCode").data_coverage_sum.count()
thresholds = [1, 3, 7, 14, 30, 365, 5 * 365]
values = [(counts_series > t).sum() for t in thresholds]
counts = [(counts_series > t).sum() for t in thresholds]


sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))
ax = sns.barplot(x=[str(t) for t in thresholds], y=counts, palette="viridis")
for i, count in enumerate(counts):
    ax.text(i, count + max(counts) * 0.01, str(count), ha='center', va='bottom', fontsize=12)
plt.xlabel("Threshold")
plt.ylabel("Number of rows with count > threshold")
plt.title("Rows with counts above various thresholds")
plt.show()

In [13]:
daily_stats = []
for idx, row in df.groupby(["healthCode", "date"]):
    data_coverage_sum = row.data_coverage.sum()
    data_coverage_mean = row.data_coverage.mean()
    n_above_zero = row[row.data_coverage > 0]
    daily_stats.append({
        "data_coverage_sum": data_coverage_sum,
        "data_coverage_mean": data_coverage_mean,
        "n_above_zero": len(n_above_zero),
        "healthCode": row["healthCode"].iloc[0]
    })

In [ ]:
data_coverage_mean

In [ ]:
len(daily_stats)

In [ ]:
daily_stats

In [5]:
import pickle
# with open("../summary_daily_v2.pkl", "wb") as f:
#     pickle.dump(daily_stats, f)
with open("../summary_daily_v2.pkl", "rb") as f:
   daily_stats = pickle.load(f)

In [ ]:
coverage_df.groupby("healthCode").data_coverage_sum.count()

In [14]:
info_df = pd.read_parquet('../myheart-counts-client/mhc_full_participant_info.parquet')

In [15]:
healthCodes_open_sharing = set(info_df[info_df.sharingScope == "all_qualified_researchers"].healthCode)

In [ ]:
coverage_df = pd.DataFrame(daily_stats)
len(coverage_df), coverage_df.data_coverage_sum.quantile(0.25)

In [ ]:
coverage_df[(coverage_df.data_coverage_sum > coverage_df.data_coverage_sum.quantile(0.25)) & (coverage_df.n_above_zero > 4)]

In [16]:
coverage_df = coverage_df[coverage_df.healthCode.isin(healthCodes_open_sharing)]

In [ ]:
coverage_df.data_coverage_sum.quantile(0.25)

## Days with ANY data

In [ ]:
len(coverage_df[coverage_df.data_coverage_sum > 0])

In [ ]:
coverage_df[coverage_df.data_coverage_sum > 0].healthCode.unique().shape

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

thresholds = [2, 3, 5, 8, 10, 15, 24]
counts = [len(coverage_df[coverage_df.n_above_zero >= t]) for t in thresholds]

sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))
ax = sns.barplot(x=[str(t) for t in thresholds], y=counts, palette="viridis")
for i, count in enumerate(counts):
    ax.text(i, count + max(counts)*0.01, str(count), ha='center', va='bottom', fontsize=12)
plt.xlabel("Threshold")
plt.ylabel("Number of rows with at least k number of modalities")
#plt.title("Rows with n_above_zero above various thresholds")
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

thresholds = [2, 3, 5, 8, 10, 15, 24]
unique_counts = [coverage_df[coverage_df.n_above_zero > t]['healthCode'].nunique() for t in thresholds]

sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))
ax = sns.barplot(x=[str(t) for t in thresholds], y=unique_counts, palette="viridis")
for i, count in enumerate(unique_counts):
    ax.text(i, count + max(unique_counts)*0.01, str(count), ha='center', va='bottom', fontsize=12)
plt.xlabel("Threshold")
plt.ylabel("Unique healthCode count")
plt.title("Unique healthCodes for rows with n_above_zero > threshold")
plt.show()

In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict
from typing import List 
from tqdm.auto import tqdm # Optional: for progress bar

def calculate_standardization_from_files(metadata_filepaths: List[str]) -> pd.DataFrame:
    """
    Calculates the overall mean and standard deviation for each feature 
    by aggregating statistics from a provided list of metadata files.

    The metadata files are expected to have been created by the 
    `create_dataset` function (or similar), containing columns 'n', 'sum', 
    'sum_of_squares', and an index representing the feature identifier 
    (expected to be named 'feature_index').

    Args:
        metadata_filepaths: A list of strings, where each string is the full 
                             path to a metadata file (e.g., a .parquet file).

    Returns:
        A pandas DataFrame indexed by 'feature_index' with columns 'mean' 
        and 'std_dev' representing the calculated standardization parameters 
        across all provided files. Returns an empty DataFrame if the list 
        is empty, no files could be processed, or they lack the required 
        columns/index.
    """
    if not metadata_filepaths:
        print("Warning: Received an empty list of metadata file paths.")
        return pd.DataFrame(columns=['mean', 'std_dev'])

    # Use defaultdict to easily accumulate sums per feature index
    aggregated_stats = defaultdict(lambda: {'total_n': 0.0, 'total_sum': 0.0, 'total_sum_of_squares': 0.0})

    print(f"Processing {len(metadata_filepaths)} metadata files. Aggregating statistics...")
    # Use tqdm if installed for a progress bar, otherwise just iterate
    file_iterator = tqdm(metadata_filepaths) if 'tqdm' in globals() else metadata_filepaths
    
    for file_path in file_iterator:
        try:
            df = pd.read_parquet(file_path)

            # Verify necessary columns and index name
            required_cols = {'n', 'sum', 'sum_of_squares'}
            if not required_cols.issubset(df.columns):
                print(f"Warning: Skipping {file_path}. Missing required columns: {required_cols - set(df.columns)}")
                continue
                
            # Check for 'feature_index' either as index name or column
            if df.index.name != 'feature_index':
                 if 'feature_index' in df.columns:
                     # Promote 'feature_index' column to be the index
                     df = df.set_index('feature_index')
                 else:
                    # Try using the existing index if it's unnamed, hoping it's the feature index
                    if df.index.name is None:
                        print(f"Warning: Index name in {file_path} is not 'feature_index'. Assuming the unnamed index represents features.")
                        df.index.name = 'feature_index' # Assign the expected name
                    else:
                        print(f"Warning: Skipping {file_path}. Index name is '{df.index.name}' (expected 'feature_index') and column not found.")
                        continue

            # Ensure data types are appropriate for summation
            try:
                df[list(required_cols)] = df[list(required_cols)].astype(float)
            except ValueError as e:
                 print(f"Warning: Skipping {file_path}. Could not convert required columns to float: {e}")
                 continue


            # Iterate through features in the current file and add to totals
            for feature_idx, row in df.iterrows():
                 # Check for NaN values in stats before aggregating
                 if pd.isna(row['n']) or pd.isna(row['sum']) or pd.isna(row['sum_of_squares']):
                     # Optionally print a warning about NaN stats, or just skip
                     # print(f"Warning: Found NaN statistics for feature {feature_idx} in {file_path}. Skipping this row.")
                     continue
                 
                 aggregated_stats[feature_idx]['total_n'] += row['n']
                 aggregated_stats[feature_idx]['total_sum'] += row['sum']
                 aggregated_stats[feature_idx]['total_sum_of_squares'] += row['sum_of_squares']

        except FileNotFoundError:
            print(f"Error: File not found {file_path}. Skipping.")
            continue
        except Exception as e:
            print(f"Error processing {file_path}: {e}. Skipping.")
            continue # Skip to the next file on error

    print("Aggregation complete. Calculating final standardization parameters...")
    
    results = []
    for feature_idx, stats in aggregated_stats.items():
        total_n = stats['total_n']
        total_sum = stats['total_sum']
        total_sum_of_squares = stats['total_sum_of_squares']

        if total_n > 0:
            mean = total_sum / total_n
            # Ensure variance is non-negative due to potential floating point issues
            # Also handle cases where sum_of_squares might be slightly less than sum^2/n due to precision
            variance_raw = (total_sum_of_squares / total_n) - (mean ** 2)
            variance = max(0, variance_raw) 
            std_dev = np.sqrt(variance)
            
            # Optional: Add a check for extremely small variance close to zero
            # if variance < 1e-10: # Adjust tolerance as needed
            #     std_dev = 0.0 
                
        else:
            # Handle cases with no valid data points for a feature
            mean = np.nan 
            std_dev = np.nan
            print(f"Warning: Feature index {feature_idx} had total_n = 0 across all processed files.")

        results.append({'feature_index': feature_idx, 'mean': mean, 'std_dev': std_dev})

    if not results:
        print("No statistics were aggregated successfully from the provided files.")
        return pd.DataFrame(columns=['mean', 'std_dev'])

    # Create final DataFrame
    final_params_df = pd.DataFrame(results)
    final_params_df = final_params_df.set_index('feature_index')
    final_params_df = final_params_df.sort_index() # Sort by feature index for consistency

    print("Standardization parameters calculated.")
    return final_params_df

files = glob.glob("/scratch/users/schuetzn/data/mhc_dataset/*/metadata.parquet")
stat_df = calculate_standardization_from_files(files)
stat_df.to_parquet("standardization_params.parquet")

In [ ]:
import pandas as pd
df = pd.read_parquet("/home/users/schuetzn/MHC_Dataset/standardization_params.parquet")
df